In [1]:
from metadrive.envs.metadrive_env import MetaDriveEnv
from metadrive.manager.traffic_manager import PGTrafficManager
from metadrive.manager.pg_map_manager import PGMapManager
from metadrive.utils.config import Config
from metadrive.examples.ppo_expert.torch_expert import torch_expert as expert
from metadrive.component.map.base_map import BaseMap
from metadrive.component.map.pg_map import MapGenerateMethod

import logging
import mediapy
from utils.scenario_runner import ScenarioRunner
from pathlib import Path
import numpy as np

In [2]:
def get_default_config() -> dict:

    # ===== Termination Scheme =====
    termination_sceme = dict(
        out_of_route_done=False,
        on_continuous_line_done=False,
        crash_vehicle_done=False,
        crash_object_done=False,
        crash_human_done=False,
    )
    # ===== Map Config =====
    map_config = {
        BaseMap.GENERATE_TYPE: MapGenerateMethod.BIG_BLOCK_NUM,
        BaseMap.GENERATE_CONFIG: 5,  # 20 block
    }

    return dict(
        # use_render=True,
        log_level=logging.INFO,  # logging.DEBUG
        traffic_density=0.1,
        traffic_mode="respawn",
        random_traffic=False,
        map_config=map_config,
        **termination_sceme,
    )

In [3]:
class MultiFidelityEnv(MetaDriveEnv):

    def __init__(self, fps=10, seed=0, config=dict()):
        self.fps = fps
        config = config | {"start_seed": seed} | get_default_config()
        super(MultiFidelityEnv, self).__init__(config)

    def _post_process_config(self, config):
        config = super(MultiFidelityEnv, self)._post_process_config(config)
        config["physics_world_step_size"] = 1 / self.fps / config["decision_repeat"]

        return config

    def get_horizon(self):
        distance = self.agent.navigation.total_length
        V_min = 2.0  # [m/s]  # set minimal velocity to 2m/s
        max_time = distance / V_min  # [s] maximum time allowed to reach the destination
        max_steps = round(max_time * self.fps)  # maximum number of simulation steps frames
        return max_steps

    def setup_engine(self):
        super(MetaDriveEnv, self).setup_engine()
        self.engine.register_manager("map_manager", PGMapManager())
        self.engine.register_manager("traffic_manager", PGTrafficManager())

In [4]:
from metadrive.engine.logger import get_logger

log = get_logger()
env = MultiFidelityEnv(fps=60, seed=10)
c = env.config
dt = env.config["physics_world_step_size"]
dr = c["decision_repeat"]
assert 1 / (dr * dt) == env.fps
log.info(f"Fidelity: {env.fps=} {dr=} {dt=:.2g}")

[INFO] Environment: MultiFidelityEnv
[INFO] MetaDrive version: 0.4.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Fidelity: env.fps=60 dr=5 dt=0.0033


In [ ]:
def get_frames(fps):
    env = MultiFidelityEnv(fps)
    _, reset_info = env.reset()
    frames = []
    for i in range(6000):
        action = expert(env.agent, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)

        frame = env.render(
            mode="topdown",
            window=False,
            screen_size=(1080 // 2, 1920 // 2),
            camera_position=env.current_map.get_center_point(),
            scaling=2,
            draw_contour=True,
            num_stack=1,
            # screen_record=True,
        )
        frames.append(frame)
        if terminated or truncated:
            break
    env.close()
    return frames

In [6]:
frames_a = get_frames(10)
frames_b = get_frames(20)

[INFO] Environment: MultiFidelityEnv
[INFO] MetaDrive version: 0.4.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 1
[INFO] Use Torch PPO expert.
[INFO] Environment: MultiFidelityEnv
[INFO] MetaDrive version: 0.4.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 1


In [7]:
mediapy.set_show_save_dir("data")

mediapy.show_videos([frames_a, frames_b[::2]], titles=["10FPS", "20FPS"])

10FPS This browser does not support the video tag.,20FPS This browser does not support the video tag.


In [8]:
env = MultiFidelityEnv(10)

[INFO] Environment: MultiFidelityEnv
[INFO] MetaDrive version: 0.4.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None


In [9]:
_ = env.reset()
action = expert(env.agent, deterministic=True)
obs, reward, terminated, truncated, info = env.step(action)

[INFO] Assets version: 0.4.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 1


In [10]:
# env.current_map.get_meta_data()
# env.engine.traffic_manager._traffic_vehicles
env.current_map.road_network.get_bounding_box()

(np.float64(-125.52134858330791),
 np.float64(121.38497161865234),
 np.float64(-399.76521435374445),
 np.float64(22.25))